# Script for solving the 1D diffusion equation
We will solve for the cooling of hot dike that was emplaced into cooler country rock. The method we are using is a simple explicit 1D finite differences scheme.

First we load some useful libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

Next we define some physics constants

In [2]:
L       = 100           # Length of modeled domain [m]
Tmagma  = 1200          # Temperature of magma [C]
Trock   = 300           # Temperature of country rock [C] 
kappa   = 1e-6          # Thermal diffusivity of rock [m2/s] 
W       = 5;            # Width of dike [m]
day     = 60*60*24       # # seconds per day
dt      = 1*day         # Timestep [s]

and some numerical constants constants

In [3]:
nx      = 201                                          # Number of gridpoints in x-direction 
nt      = 500                                          # Number of timesteps to compute
Xvec,dx = np.linspace(-L/2, L/2, nx,  retstep=True)    # X coordinate vector, constant spacing
beta    = kappa*dt/dx**2

Set the initial conditions

In [4]:
T       = np.ones((nt, nx))*Trock;              # everything is cold initially
T[0,np.nonzero(np.abs(Xvec) <= W/2)] = Tmagma   # and hot where the dike is
time    = 0                                     # track the run time
T_init=T[0,:]

Make a function that computes temperature

In [5]:
# option 1: store all time steps solution to T
def fdm_solve(T):
      
    for n in range(0, nt-1, 1):
        for i in range(1,len(Xvec)-1):
            T[n + 1, i] = beta * (T[n][i+1] - 2*T[n][i] + T[n][i-1]) + T[n][i]

    return T

# option 2: only store the latest time step solution to Told 
def fdm_solve2(Told):
    Tnew=Told.copy()
    for i in range(1,len(Xvec)-1):
        Tnew[i] = beta * (Told[i+1] - 2*Told[i] + Told[i-1]) + Told[i]
    return Tnew

and solve for temperature

In [6]:
T = fdm_solve(T)

finally we visualize everything

In [7]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(10,5))
ax = plt.axes(xlim=(-L/2, L/2), ylim=(0, Tmagma))
line, = ax.plot([], [], lw=1)
timeLabel=ax.text(0.02,0.98,'Time: ',transform=ax.transAxes,va='top')
ax.set_xlabel('X (m)')
ax.set_ylabel('Temperature ($^{\circ}$C)')

# Initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# Initialize Tnew
Tnew=T_init
# Animation function which updates figure data.  This is called sequentially
def animate(i):
    timeLabel._text='Time: %.1f day'%(i*dt/day)
    
    # option1: 
    #line.set_data(Xvec, T[i,:])
    
    # option 2: use global keyword to store the latest solution and update it using fdm_solve2 function
    global Tnew
    Tnew=fdm_solve2(Tnew)
    line.set_data(Xvec, Tnew)
    
    return line,

# Call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=nt, interval=30, blit=True)

plt.close(anim._fig)

# Call function to display the animation
HTML(anim.to_html5_video())  # lower resolution
# HTML(anim.to_jshtml())  # higher resolution